In [94]:
%config IPCompleter.use_jedi = False

In [95]:
import pandas as pd
import numpy as np
import math

In [96]:
parties_df_raw = pd.read_html('https://ru.wikipedia.org/wiki/%D0%9F%D0%B0%D1%80%D0%BB%D0%B0%D0%BC%D0%B5%D0%BD%D1%82%D1%81%D0%BA%D0%B8%D0%B5_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D1%8B_%D0%B2_%D0%91%D0%BE%D0%BB%D0%B3%D0%B0%D1%80%D0%B8%D0%B8_(2017)')[4]

In [97]:
parties_df_raw = parties_df_raw.iloc[:5]

In [98]:
election_dict = {x[0]:int(x[1]) for x in list(parties_df_raw[['Партия',
                                                         'Мест']].\
                                         set_index('Партия').\
                                         to_records())}

In [99]:
election_dict

{'ГЕРБ': 95,
 'БСП за Болгарию': 80,
 'Объединённые патриоты': 27,
 'Движение за права и свободы': 26,
 'Воля': 12}

In [100]:
assert sum(list(election_dict.values()))==240
# проверка

In [101]:
QUOTA = math.ceil((240)/2)+1

In [102]:
QUOTA

121

In [103]:
from itertools import combinations

# https://stackoverflow.com/a/24402296
combinations = sum([list(map(list, combinations(election_dict, i))) for i in range(len(election_dict) + 1)], [])

In [104]:
combinations_clean = [combination for combination in combinations if len(combination)>1]

In [105]:
def get_vote_sum(combination):
    vote_sum = 0
    for party in combination:
        vote_sum += election_dict[party]
    return vote_sum

In [106]:
winning_coalitions = [combination for combination in combinations_clean if get_vote_sum(combination)>=QUOTA]

In [107]:
def is_pivotal(party, coalition):
    if party in coalition:
        sum_wout_party = sum([election_dict[element] for element in [prt for prt in coalition if prt != party]])
        if sum_wout_party < QUOTA:
            return True
        else:
            return False
    else:
        return False

In [108]:
for party in list(election_dict.keys()):
    res_list = []
    for coalition in winning_coalitions:
        res_list.append(is_pivotal(party, coalition))
    print(party, sum(res_list))

ГЕРБ 12
БСП за Болгарию 4
Объединённые патриоты 4
Движение за права и свободы 4
Воля 0


In [109]:
pivotal = {
    'ГЕРБ': 12,
    'БСП за Болгарию': 4,
    'Объединённые патриоты': 4,
    'Движение за права и свободы': 3,
    'Воля': 0
}

In [110]:
sum_pivot = sum(list(pivotal.values()))

In [111]:
def banzhaf(party, pivotal):
    return(pivotal[party]/sum(list(pivotal.values())))
    print(party)

In [112]:
indices = []
for party in list(election_dict.keys()):
    banzhaf(party, pivotal)
    indices.append(banzhaf(party, pivotal))

In [113]:
sum(indices)

1.0

In [114]:
def shapley_shubik(party, coalition):
    global election_dict
    if party in coalition:
        if is_pivotal(party, coalition):
            n = (len(list(election_dict.keys())))
            s = len(coalition)
            # print(n, s)
            return (math.factorial(n-s)*math.factorial(s-1))/math.factorial(n)
        else:
            return 0
    else:
        return 0

In [115]:
indices = []

for party in list(election_dict.keys()):
    party_index = 0
    for coalition in winning_coalitions:
        party_index += shapley_shubik(party, coalition)
    print(party, party_index)
    indices.append(party_index)

ГЕРБ 0.49999999999999994
БСП за Болгарию 0.16666666666666669
Объединённые патриоты 0.16666666666666669
Движение за права и свободы 0.16666666666666669
Воля 0


In [116]:
sum(indices)

1.0